# Does the Quantum Ensemble outperforms the single Quantum Cosine Classifier?

In [ ]:
import sys
sys.path.insert(1, '../')

from Utils import *
from modeling import *

seed = 54321 
n_shots = 8192

# number of control qubits => 2^d is the ensemble size
d=0

# number of training point to classify a single test point
n_train=1 

# constant value on the diagonal covariance matrix
std=.3 

n = 200 # dataset size
test_size = .10 # test size

In [ ]:
X, y = load_data(n=n, std=std, plot=True, save=False)

# Preprocessing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123, test_size=test_size)

Y_vector_train = label_to_array(y_train)
Y_vector_test = label_to_array(y_test)

In [ ]:
predictions = []

np.random.seed(seed)
for x_test, y_ts in zip(X_test, Y_vector_test):
    ix = np.random.choice(int(n*(1-test_size)), 1)[0]
    x_train = X_train[ix]
    x_tr = normalize_custom(x_train)
    y_tr = Y_vector_train[ix]
    x_ts = normalize_custom(x_test)

    qc = cos_classifier(x_tr, x_ts, y_tr)
    r = exec_simulator(qc, n_shots=n_shots)

    if '0' not in r.keys():
        r['0'] = 0
    elif '1' not in r.keys():
        r['1'] = 0

    predictions.append(retrieve_proba(r))

a, b = evaluation_metrics(predictions, X_test, y_test, save=False)

print('seed:', seed, '   Accuracy:', a, '   Brier score:', b)

## Load IBMQ account

In [ ]:
# IBMQ.load_account()
# provider = IBMQ.get_provider(hub='ibm-q')
# provider.backends()
# backend = provider.get_backend('ibmq_qasm_simulator')

In [ ]:
np.random.seed(seed)
n_swap = 1
balanced = True

def run_ensemble(d, n_train, seed, n_swap, balanced):    
    accuracy = []; predictions = []

    np.random.seed(seed)
    for x_test, y_ts in zip(X_test, Y_vector_test):
        X_data, Y_data = training_set(X_train, y_train, n=n_train, seed=seed)
        x_test = normalize_custom(x_test)

        qc = ensemble(X_data, Y_data, x_test, n_swap=n_swap, d=d, balanced=True)
        qc = transpile(qc, basis_gates=['u1', 'u2', 'u3', 'cx'], optimization_level=1)

        # job = execute(qc, backend, shots=n_shots)
        # results = job.result()
        # r = results.get_counts(qc)
        r = exec_simulator(qc, n_shots=n_shots)

        predictions.append(retrieve_proba(r))
        # print(retrieve_proba(r), y_ts)

    a, b = evaluation_metrics(predictions, X_test, y_test, save=False)
    print('seed:', seed, '   std:', std, '   d:', d, '   Accuracy:', a, '   Brier score:', b)

## Running Ensemble

In [ ]:
vector_d = [1,2]
vector_n = [2,4]

for d, n_train in zip(vector_d, vector_n):
    run_ensemble(d, n_train, seed, n_swap, balanced)

# Results on the paper

<center><img src="IMG/results_ensemble.JPG" width="80%" height="80%"/></center>

<center><img src="IMG/results_ensemble2.JPG" width="80%" height="80%"/></center>